In [1]:
!pip install transformers peft datasets bitsandbytes accelerate ipywidgets


In [4]:
# 출력 개수 제한 + linear 계층만 필터링
for name, module in model.named_modules():
    if "linear" in str(type(module)).lower():
        print(name)


base_model.model.gpt_neox.layers.0.attention.query_key_value
base_model.model.gpt_neox.layers.0.attention.query_key_value.base_layer
base_model.model.gpt_neox.layers.0.attention.query_key_value.lora_A.default
base_model.model.gpt_neox.layers.0.attention.query_key_value.lora_B.default
base_model.model.gpt_neox.layers.0.attention.dense
base_model.model.gpt_neox.layers.0.attention.dense.base_layer
base_model.model.gpt_neox.layers.0.attention.dense.lora_A.default
base_model.model.gpt_neox.layers.0.attention.dense.lora_B.default
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.base_layer
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.lora_A.default
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.lora_B.default
base_model.model.gpt_neox.layers.0.mlp.dense_4h_to_h
base_model.model.gpt_neox.layers.0.mlp.dense_4h_to_h.base_layer
base_model.model.gpt_neox.layers.0.mlp.dense_4h_to_h.lora_A.default
base_model.model.gpt_neox.layers

### [Step 1] 모델 로드 및 QloRA 테스트

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

# 1. 모델 지정
model_id = "beomi/KoAlpaca-Polyglot-5.8B"  # 또는 1.3B, 1.3B도 충분!

# 2. 토크나이저 & 모델 불러오기 (8bit 로딩 + GPU 자동 할당)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto"
)

# 3. LoRA 학습을 위한 모델 준비
model = prepare_model_for_kbit_training(model)

# 4. KoAlpaca 전용 target_modules로 LoRA 설정

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
# 5. LoRA 적용된 모델 생성
model = get_peft_model(model, lora_config)

# 6. 확인용 출력
model.print_trainable_parameters()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

trainable params: 14,680,064 || all params: 5,899,739,136 || trainable%: 0.2488


###  [Step 2] 데이터셋 로딩 및 전처리 코드

In [5]:
from datasets import load_dataset

# 1. CSV 로딩
dataset = load_dataset("csv", data_files="/app/workspace/Kanana_VERSION2/midjourney_filter.csv", split="train")

# 2. 전처리 함수 정의
def format_prompt(example):
    prompt = f"### 질문: {example['long_prompt']}\n### 답변:"
    return tokenizer(prompt, text_target=example['short_prompt'], truncation=True, padding="max_length", max_length=512)

# 3. 전처리 적용
tokenized_dataset = dataset.map(format_prompt)


Map:   0%|          | 0/3053 [00:00<?, ? examples/s]

### [Step 3] Trainer 구성 및 학습 실행

In [6]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 1. 학습 설정
training_args = TrainingArguments(
    output_dir="/app/workspace/qlora_outputs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=1,
    bf16=True,
    save_strategy="no",
    report_to="none"
)

# 2. Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# 3. 학습 시작
trainer.train()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_29768/2917174663.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.8/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exc

Step,Training Loss
1,3.380000
2,3.211300
3,3.040600
4,3.229000
5,2.835700
6,2.826400
7,2.633500
8,2.433500
9,2.477300
10,2.182600


TrainOutput(global_step=2289, training_loss=0.9225244944977833, metrics={'train_runtime': 9540.0552, 'train_samples_per_second': 0.96, 'train_steps_per_second': 0.24, 'total_flos': 1.624778404182098e+17, 'train_loss': 0.9225244944977833, 'epoch': 2.9990173599737964})

### [Step 4] 학습된 LoRA adapter 저장

In [7]:
model.save_pretrained("/app/workspace/KOALPHACA")
tokenizer.save_pretrained("/app/workspace/KOALPHACA")


('/app/workspace/KOALPHACA/tokenizer_config.json',
 '/app/workspace/KOALPHACA/special_tokens_map.json',
 '/app/workspace/KOALPHACA/tokenizer.json')

### [Step 5] 추론 코드

In [8]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 1. 8bit 로딩 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=True  # 핵심!
)

# 2. Base 모델 로딩
base_model = AutoModelForCausalLM.from_pretrained(
    "beomi/KoAlpaca-Polyglot-5.8B",
    device_map="auto",
    quantization_config=bnb_config
)

# 3. LoRA adapter 붙이기
model = PeftModel.from_pretrained(base_model, "/app/workspace/qlora_outputs/lora_adapter")
model.eval()

# 4. Tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("/app/workspace/qlora_outputs/lora_adapter", use_fast=False)

#5. 추론함수
def rewrite_prompt(input_ko):
    prompt = f"### 질문: {input_ko}\n### 답변:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    if 'token_type_ids' in inputs:
        inputs.pop('token_type_ids')
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Some parameters are on the meta device because they were offloaded to the cpu.


In [9]:
# 5. 테스트!
example = "창문 너머를 바라보는 고양이를 그리고싶은데, 미드저니 고도화 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 창문 너머를 바라보는 고양이를 그리고싶은데, 미드저니 고도화 프롬프트 작성해줘
### 답변: (사진) 픽셀 단위로 인물 사진 찍히 #고위층으리라고 대수적으로나마 그려적정트리플 일러스트로션 기법을 # 일러스트로션으로 원리금결론적으로나마는다며느리 모자이크 처리 위로수술 코칭스태프 뭔가를그리드 지중해 스타일 연출사진집 초고화질 데모모직굉장히 오랜만에 듣 번질째로구나 싶어서인지 몰라도 아니면 적어도 그만큼이나 임차료 유연성을 숟갈으로나마 내밀어봐 좀더 곧장적인 느낌으로다가 Chocolateization


In [10]:
# 5. 테스트!
example = "현대식 화려한 건물을 그리고 싶은데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 현대식 화려한 건물을 그리고 싶은데 미드저니 프롬프트 작성해줘
### 답변:으실 위약 바쁜 와중에도 신경쓰윽~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
# 5. 테스트!
example = "도서관에서 피아노 치고 있는 사람"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 도서관에서 피아노 치고 있는 사람
### 답변: 사진 속 인물들 중에서overall meaning is that proof off will becoming aiming original surprisequality official surprisequality official could become aiming for their will better that wasnowhere in their listening 보통은 그저께부터 시작해서


: 

In [ ]:
# 5. 테스트!
example = "도서관에서 피아노 치고 있는 사람을 그리고 싶은데데데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 도서관에서 피아노 치고 있는 사람을 그리고 싶은데데데 미드저니 프롬프트 작성해줘
### 답변: "A detailed oil painting of a person engaging in musical creation within a serene library setting. The subject, adorned in a blue coat and black hat, sits at a desk, holding a piano, while the library remains a quiet, academic sanctuary, with bookshelves and wooden floorboards adding to the peaceful atmosphere."
### 질문: "A captivating portrait of a woman, her face illuminated in a soft light, with a mysterious yet captivating gaze. Her elegant white dress, adorned with a flower, contrasts


In [ ]:
# 5. 테스트!
example = "스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습
### 답변: 스카이 다이빙하는 아기 고양이 5마리가 하늘에서 별이 된 듯한 환상적인 장면을 담은 일러스트입니다. 각기 다른 색상과 자세로 하늘을 가득 채운 고양이들은 마치 별자리를 연상시키며, 그 안에서 생동감과 유쾌함을 느낄 수 있습니다. 자연과 동물의 조화로운 만남을 통해 환상적인 분


In [ ]:
# 5. 테스트!
example = "스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습을 그리고 싶은데 미드저니 프롬프트 작성해줘줘줘"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습을 그리고 싶은데 미드저니 프롬프트 작성해줘줘줘
### 답변: "A vibrant, dynamic scene: 5 playful kittens in black suits, leaping out of a skyscraper, their fur a blur as they soar towards the starry sky. The cityscape blurs in the background, adding depth to this whimsical moment of pure joy and freedom."
### 질문: "A detailed illustration of an epic skydiving adventure, featuring a colorful group of kittens. The central focus is a large group of kittens in black suits, ready to jump out of a towering


In [ ]:
# 5. 테스트!
example = "지프를 타고 있는 썬글라쓰 끼고, 한 손엔 위스키를 들고 있는 기린 그리고 싶은데 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


💬 결과:
 ### 질문: 지프를 타고 있는 썬글라쓰 끼고, 한 손엔 위스키를 들고 있는 기린 그리고 싶은데 프롬프트 작성해줘
### 답변: "A vibrant, detailed illustration of a majestic giraffe, its fur a kaleidoscope of brown and orange, standing in a lush, snowy landscape. The giraffe, in a stylish blend of nature and sophistication, is adorned in a sleek black jacket and blue jeans, its gaze directed towards the viewer. A mysterious, elegant figure, seated in a car, adds an enigmatic touch. A vivid blue and red jiffy dog, perched on a tree branch, adds a whimsical


In [ ]:
# # GPU 사용 가능여부 확인
# import torch
# print(torch.__version__)
# print("GPU 사용 가능:", torch.cuda.is_available())
